In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path_to_invasive = '/content/drive/MyDrive/SRCNN/output'
path_to_non_invasive = '/content/drive/MyDrive/SRCNN/output_non'


In [1]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

def load_images_and_labels(image_paths, label):
    images = []
    labels = []
    try:
        file_names = os.listdir(image_paths)
        if not file_names:
            print("Directory is empty:", image_paths)
        for image_path in file_names:
            full_path = os.path.join(image_paths, image_path)
            print("Loading:", full_path)  # Debug: Print the path of each image being loaded
            img = load_img(full_path, target_size=(32, 32))
            img = img_to_array(img)
            images.append(img)
            labels.append(label)
    except Exception as e:
        print(f"Error loading images: {e}")
    return np.array(images), np.array(labels)



x_invasive, y_invasive = load_images_and_labels(path_to_invasive, 1)
x_non_invasive, y_non_invasive = load_images_and_labels(path_to_non_invasive, 0)


NameError: name 'path_to_invasive' is not defined

In [1]:
from sklearn.model_selection import train_test_split

# Concatenate invasive and non-invasive data
x = np.concatenate((x_invasive, x_non_invasive), axis=0)
y = np.concatenate((y_invasive, y_non_invasive), axis=0)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


NameError: name 'np' is not defined

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [ ]:
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

In [ ]:
num_classes = 2
input_shape = (32, 32, 3)

In [ ]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 10
dropout_rate = 0.5
temperature = 0.05

In [ ]:
from keras.losses import CategoricalCrossentropy

def create_classifier(encoder, trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=CategoricalCrossentropy(),  # Changed here
        metrics=['accuracy']  # Simplified metric for clarity
    )
    return model

# Create the encoder and classifier again
encoder = create_encoder()
classifier = create_classifier(encoder)

# Now fit the model
history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

# Evaluate the model
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Epoch 1/50
266/266 [==============================] - 4233s 16s/step - loss: 0.2641 - accuracy: 0.9030
Epoch 2/50
162/266 [=================>............] - ETA: 26:44 - loss: 0.1680 - accuracy: 0.9387

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

class SupervisedContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=0.05):
        super().__init__()
        self.temperature = temperature

    def call(self, labels, feature_vectors):
        # Normalize the feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.matmul(feature_vectors_normalized, feature_vectors_normalized, transpose_b=True) / self.temperature
        # Check shapes
        print("Logits shape:", logits.shape)
        print("Labels shape:", labels.shape)
        # Flatten labels to ensure they are one-dimensional
        labels = tf.reshape(labels, [-1])
        return tfa.losses.npairs_loss(labels, logits)

def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="with_projection-head"
    )
    return model

In [ ]:
encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder)

# Assuming your model's projection head outputs embeddings correctly
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature)
)

encoder_with_projection_head.summary()




In [ ]:

y_train = y_train.astype('int32')
print(y_train.dtype)  # This should output 'int32' or a similar integer type

In [ ]:
if y_train.ndim > 1:
    y_train = tf.argmax(y_train, axis=1)

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)


In [ ]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

In [ ]:
# If `y_test` is one-hot encoded, convert it to class indices
y_test_indices = tf.argmax(y_test, axis=1)

# Evaluate using the corrected labels
accuracy = classifier.evaluate(x_test, y_test_indices)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


87